Переупорядочиваем SNP

In [4]:
!plink --bfile ./penncath_data/penncath --make-bed --out intermediate_penncath

Python(35428) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


PLINK v1.9.0-b.7.7 64-bit (22 Oct 2024)            cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to intermediate_penncath.log.
Options in effect:
  --bfile ./penncath_data/penncath
  --make-bed
  --out intermediate_penncath

16384 MB RAM detected; reserving 8192 MB for main workspace.
861473 variants loaded from .bim file.
1401 people (937 males, 464 females) loaded from .fam.
933 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1401 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.977517.
861473 variants and 1401 people pass filters and QC.
Among remaining phenotypes, 463 are cases and 470 are controls.  (468
phenotyp

Фильтруем SNP

In [24]:
!plink --bfile intermediate_penncath --maf 0.05 --geno 0.05 --hwe 1e-6 --make-bed --out filtered_data

Python(11693) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


PLINK v1.9.0-b.7.7 64-bit (22 Oct 2024)            cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to filtered_data.log.
Options in effect:
  --bfile intermediate_penncath
  --geno 0.05
  --maf 0.05
  --make-bed
  --out filtered_data

16384 MB RAM detected; reserving 8192 MB for main workspace.
861473 variants loaded from .bim file.
1401 people (937 males, 464 females) loaded from .fam.
933 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1401 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.977517.
94293 variants removed due to missing genotype data (--geno).
175182 variants removed due to minor allele threshold(s)
(--m

Замена фенотипа на таргет из csv файла.

In [25]:
import pandas as pd
import numpy as np
import subprocess
import os
from sklearn.model_selection import KFold


In [26]:
fam = pd.read_csv('./filtered_data.fam', sep=' ', header=None)
csv = pd.read_csv('./penncath_data/penncath.csv', sep=',')
fam[5] = csv["CAD"] + 1
fam.to_csv("filtered_data_1.fam", 
           index=False,
           sep=' ',
           header=False
)
fam, csv

(          0  1  2  3  4  5
 0     10002  1  0  0  1  2
 1     10004  1  0  0  2  2
 2     10005  1  0  0  1  2
 3     10007  1  0  0  1  2
 4     10008  1  0  0  1  2
 ...     ... .. .. .. .. ..
 1396  11591  1  0  0  2  1
 1397  11592  1  0  0  1  2
 1398  11593  1  0  0  1  2
 1399  11594  1  0  0  1  2
 1400  11596  1  0  0  1  1
 
 [1401 rows x 6 columns],
       FamID  CAD  sex  age     tg    hdl    ldl
 0     10002    1    1   60    NaN    NaN    NaN
 1     10004    1    2   50   55.0   23.0   75.0
 2     10005    1    1   55  105.0   37.0   69.0
 3     10007    1    1   52  314.0   54.0  108.0
 4     10008    1    1   58  161.0   40.0   94.0
 ...     ...  ...  ...  ...    ...    ...    ...
 1396  11591    0    2   59   34.0   44.0   89.0
 1397  11592    1    1   45   69.0  101.0   77.0
 1398  11593    1    1   59   77.0   27.0   41.0
 1399  11594    1    1   30    NaN    NaN    NaN
 1400  11596    0    1   64  224.0   35.0   96.0
 
 [1401 rows x 7 columns])

In [27]:
fam[5].mean()

1.6659528907922911

Испрравляем формат первого столбца после записи через пандас

In [28]:
with open('filtered_data_1.fam', 'r') as infile, open('filtered_data.fam', 'w') as outfile:
    for line in infile:
        number, value, a, b, c, d = line.split()

        outfile.write(f'"{number}" {value} {a} {b} {c} {d}\n')

In [29]:
input_file = "./filtered_data"

fam_file = input_file + '.fam'
fam_data = pd.read_csv(fam_file, sep='\s+', header=None)

samples = fam_data.iloc[:, [0, 1]]


<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
/var/folders/vk/0y4fmhhn3cd_lwzv7g3n3fc5f83rjt/T/ipykernel_10568/1294136838.py:4: SyntaxWarning: invalid escape sequence '\s'
  fam_data = pd.read_csv(fam_file, sep='\s+', header=None)


In [30]:
shuffled_samples = samples.sample(frac=1, random_state=42)

In [31]:
total_samples = len(shuffled_samples)
train_size = int(total_samples * 0.8)
# Разделение на обучающую и валидационную выборки
train_samples = shuffled_samples.iloc[:train_size]
validation_samples = shuffled_samples.iloc[train_size:]
print(f"Всего: {total_samples}")
print(f"Обучающая выборка {len(train_samples)} - ({len(train_samples)/total_samples*100:.1f}%)")
print(f"Валидационная выборка {len(validation_samples)} - ({len(validation_samples)/total_samples*100:.1f}%)")

Всего: 1401
Обучающая выборка 1120 - (79.9%)
Валидационная выборка 281 - (20.1%)


In [32]:
train_file = 'train_samples.txt'
validation_file = 'validation_samples.txt'
train_file_1 = 'train_samples_1.txt'
validation_file_1 = 'validation_samples_1.txt'

train_samples.to_csv(train_file_1, sep='\t', header=False, index=False)
validation_samples.to_csv(validation_file_1, sep='\t', header=False, index=False)

In [33]:
with open('train_samples_1.txt', 'r') as infile, open('train_samples.txt', 'w') as outfile:
    for line in infile:
        number, value = line.split()

        outfile.write(f'"{number}" {value}\n')

with open('validation_samples_1.txt', 'r') as infile, open('validation_samples.txt', 'w') as outfile:
    for line in infile:
        number, value = line.split()

        outfile.write(f'"{number}" {value}\n')

In [34]:
try:
    # Создание обучающей выборки
    print("Создание обучающей выборки")
    subprocess.run([
        'plink',
        '--bfile', input_file,
        '--keep', train_file,
        '--make-bed',
        '--out', 'all_train'
    ], check=True)
    # Создание валидационной выборки
    print("Создание валидационной выборки")
    subprocess.run([
        'plink',
        '--bfile', input_file,
        '--keep', validation_file,
        '--make-bed',
        '--out', 'validation'
    ], check=True)
    print("Разделение успешно")
    
except subprocess.CalledProcessError as e:
    print(f"Ошибка {e}")

Создание обучающей выборки
PLINK v1.9.0-b.7.7 64-bit (22 Oct 2024)            cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to all_train.log.
Options in effect:
  --bfile ./filtered_data
  --keep train_samples.txt
  --make-bed
  --out all_train

16384 MB RAM detected; reserving 8192 MB for main workspace.
591998 variants loaded from .bim file.
1401 people (937 males, 464 females) loaded from .fam.
1401 phenotype values loaded from .fam.
--keep: 1120 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1120 founders and 0 nonfounders present.
Calculating allele frequencies... 1011%

Python(11787) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


1213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is 0.991709.
591998 variants and 1120 people pass filters and QC.
Among remaining phenotypes, 744 are cases and 376 are controls.
--make-bed to all_train.bed + all_train.bim + all_train.fam ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
Создание валидационной выборки
PLINK v1.9.0-b.7.7 64-bit (22 Oct 2024)            cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to validation.log.
Options in effect:
  --bfile ./filtered_data
  --keep validation_samples.txt
  --make-bed
  --out validation

16384 MB RAM detected; reserving 8192 MB for main workspace.
591998 

Python(11788) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is 0.991561.
591998 variants and 281 people pass filters and QC.
Among remaining phenotypes, 189 are cases and 92 are controls.
--make-bed to validation.bed + validation.bim + validation.fam ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
Разделение успешно


In [35]:
for file in [train_file, validation_file, train_file_1, validation_file_1]:
    if os.path.exists(file):
        os.remove(file)


In [36]:
input_file = "./all_train"

fam_file = input_file + '.fam'
fam_data = pd.read_csv(fam_file, sep='\s+', header=None)

samples = fam_data.iloc[:, [0, 1]]

samples.head()


<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
/var/folders/vk/0y4fmhhn3cd_lwzv7g3n3fc5f83rjt/T/ipykernel_10568/3204243319.py:4: SyntaxWarning: invalid escape sequence '\s'
  fam_data = pd.read_csv(fam_file, sep='\s+', header=None)


,0,1
0,10002,1
1,10005,1
2,10007,1
3,10008,1
4,10009,1


In [37]:
# Перемешиваем с фиксированным seed
np.random.seed(42)
shuffled_indices = np.random.permutation(len(samples))
shuffled_samples = samples.iloc[shuffled_indices].reset_index(drop=True)

n_folds = 5
fold_size = len(shuffled_samples) // n_folds
fold_samples = []

for i in range(n_folds):
    start_idx = i * fold_size
    end_idx = (i + 1) * fold_size if i < n_folds - 1 else len(shuffled_samples)
    fold_samples.append(shuffled_samples.iloc[start_idx:end_idx])
    print(f"Фолд {i+1}: {len(fold_samples[i])} образцов")


Фолд 1: 224 образцов
Фолд 2: 224 образцов
Фолд 3: 224 образцов
Фолд 4: 224 образцов
Фолд 5: 224 образцов


In [38]:
def create_plink_sets_for_fold(fold_idx, fold_samples, all_samples, input_file):
    """
        fold_idx: 0-4
        fold_samples: список DataFrames с обьектами для каждого фолда
        all_samples: полный DataFrame
        input_file: путь к PLINK
    """
    test_samples = fold_samples[fold_idx]
    train_indices = list(range(len(fold_samples)))
    train_indices.remove(fold_idx)
    train_samples = pd.concat([fold_samples[i] for i in train_indices])
    
    test_name = f"test_{fold_idx+1}"
    train_name = f"train_{fold_idx+1}"
    test_file = f"{test_name}_samples.txt"
    train_file = f"{train_name}_samples.txt"
    test_file_1 = f"{test_name}_samples_1.txt"
    train_file_1 = f"{train_name}_samples_1.txt"
    
    test_samples.to_csv(test_file_1, sep='\t', header=False, index=False)
    train_samples.to_csv(train_file_1, sep='\t', header=False, index=False)

    with open(test_file_1, 'r') as infile, open(test_file, 'w') as outfile:
        for line in infile:
            number, value = line.split()
            outfile.write(f'"{number}" {value}\n')
    
    with open(train_file_1, 'r') as infile, open(train_file, 'w') as outfile:
        for line in infile:
            number, value = line.split()
            outfile.write(f'"{number}" {value}\n')
    
    try:
        print(f"Создание тестового набора {test_name}")
        subprocess.run([
            'plink',
            '--bfile', input_file,
            '--keep', test_file,
            '--make-bed',
            '--out', test_name
        ], check=True)
        
        print(f"Создание обучающего набора {train_name}")
        subprocess.run([
            'plink',
            '--bfile', input_file,
            '--keep', train_file,
            '--make-bed',
            '--out', train_name
        ], check=True)
        
        print(f"Фолд {fold_idx+1}: успешно")
        for file in [test_file, train_file, test_file_1, train_file_1]:
            if os.path.exists(file):
                os.remove(file)
        return True
        
    except subprocess.CalledProcessError as e:
        print(f"Ошибка  {e}")
        for file in [test_file, train_file, test_file_1, train_file_1]:
            if os.path.exists(file):
                os.remove(file)
        return False


In [39]:
for i in range(n_folds):
    print(f"\n{'='*50}")
    print(f"фолд {i+1}/{n_folds}")
    create_plink_sets_for_fold(i, fold_samples, shuffled_samples, input_file)
    print(f"{'='*50}\n")



фолд 1/5
Создание тестового набора test_1
PLINK v1.9.0-b.7.7 64-bit (22 Oct 2024)            cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to test_1.log.
Options in effect:
  --bfile ./all_train
  --keep test_1_samples.txt
  --make-bed
  --out test_1

16384 MB RAM detected; reserving 8192 MB for main workspace.
591998 variants loaded from .bim file.
1120 people (749 males, 371 females) loaded from .fam.
1120 phenotype values loaded from .fam.
--keep: 224 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 224 founders and 0 nonfounders present.
Calculating allele frequencies... 9%

Python(11838) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is 0.991813.
591998 variants and 224 people pass filters and QC.
Among remaining phenotypes, 145 are cases and 79 are controls.
--make-bed to test_1.bed + test_1.bim + test_1.fam ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
Создание обучающего набора train_1
PLINK v1.9.0-b.7.7 64-bit (22 Oct 2024)            cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to train_1.log.
Options in effect:
  --bfile ./all_train
  --keep train_1_samples.txt
  --make-bed
  --out train_1

16384 MB RAM detected; reserving 8192 MB for main workspace.
591998 variants loaded f

Python(11840) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is 0.991683.
591998 variants and 896 people pass filters and QC.
Among remaining phenotypes, 599 are cases and 297 are controls.
--make-bed to train_1.bed + train_1.bim + train_1.fam ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
Фолд 1: успешно


фолд 2/5
Создание тестового набора test_2
PLINK v1.9.0-b.7.7 64-bit (22 Oct 2024)            cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to test_2.log.
Options in effect:
  --bfile ./all_train
  --keep test_2_samples.txt
  --make-bed
  --out test_2

16384 MB RAM detected; reserving 8192 MB for main workspace.
591998 vari

...

PLINK v1.9.0-b.7.7 64-bit (22 Oct 2024)            cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to train_4.log.
Options in effect:
  --bfile ./all_train
  --keep train_4_samples.txt
  --make-bed
  --out train_4

16384 MB RAM detected; reserving 8192 MB for main workspace.
591998 variants loaded from .bim file.
1120 people (749 males, 371 females) loaded from .fam.
1120 phenotype values loaded from .fam.
--keep: 896 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 896 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is 0.991765.
591998 variants and 896 people pass filters and QC.
Among remaining phenotypes, 595 are case

Python(11854) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


40414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is 0.992207.
591998 variants and 224 people pass filters and QC.
Among remaining phenotypes, 150 are cases and 74 are controls.
--make-bed to test_5.bed + test_5.bim + test_5.fam ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
Создание обучающего набора train_5
PLINK v1.9.0-b.7.7 64-bit (22 Oct 2024)            cog-genomics.org/plink/1.9/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to train_5.log.
Options in effect:
  --bfile ./all_train
  --keep train_5_samples.txt
  --make-bed
  --out train_5

16384 MB RAM detected; reserving 8192 MB for main workspace.
591998 variants loaded from .bim file.
1120 people (749 males, 371 females) loaded 

Python(11865) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


37383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate in remaining samples is 0.991585.
591998 variants and 896 people pass filters and QC.
Among remaining phenotypes, 594 are cases and 302 are controls.
--make-bed to train_5.bed + train_5.bim + train_5.fam ... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899done.
Фолд 5: успешно



In [40]:
# Проверяем файлы
file_types = ['.bed', '.bim', '.fam']
all_files = []

for i in range(1, n_folds+1):
    for prefix in [f'test_{i}', f'train_{i}']:
        for file_type in file_types:
            all_files.append(f"{prefix}{file_type}")

for file in all_files:
    exists = os.path.exists(file)
    if not exists:
        print(f"проблема с {file}")

for i in range(1, n_folds+1):
    test_file = f"test_{i}.fam"
    train_file = f"train_{i}.fam"
    test_count = len(pd.read_csv(test_file, sep='\s+', header=None))
    train_count = len(pd.read_csv(train_file, sep='\s+', header=None))
    print(f"Фолд {i} тест = {test_count} обучение = {train_count} всего = {test_count + train_count}")


Фолд 1 тест = 224 обучение = 896 всего = 1120
Фолд 2 тест = 224 обучение = 896 всего = 1120
Фолд 3 тест = 224 обучение = 896 всего = 1120
Фолд 4 тест = 224 обучение = 896 всего = 1120
Фолд 5 тест = 224 обучение = 896 всего = 1120


<>:18: SyntaxWarning: invalid escape sequence '\s'
<>:19: SyntaxWarning: invalid escape sequence '\s'
<>:18: SyntaxWarning: invalid escape sequence '\s'
<>:19: SyntaxWarning: invalid escape sequence '\s'
/var/folders/vk/0y4fmhhn3cd_lwzv7g3n3fc5f83rjt/T/ipykernel_10568/1896703663.py:18: SyntaxWarning: invalid escape sequence '\s'
  test_count = len(pd.read_csv(test_file, sep='\s+', header=None))
/var/folders/vk/0y4fmhhn3cd_lwzv7g3n3fc5f83rjt/T/ipykernel_10568/1896703663.py:19: SyntaxWarning: invalid escape sequence '\s'
  train_count = len(pd.read_csv(train_file, sep='\s+', header=None))
